In [155]:
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import re

In [156]:
LA_codes = ['USW00023130', 'USW00093134']

# Hexatic Order Parameter
hexOrder = pd.DataFrame()
for code in LA_codes:
    loc = 'E:/Old Downloads Folder/Research Stuff/Jupyter Notebooks/Hexatic Order Parameters/'
    s2 = '_hexatic_order_parameter.txt'
    s = loc + code + s2
    
    with open(s) as f:
        val = f.read()
        temp_df = pd.DataFrame([[code, val[4:]]], columns=['Station', 'Hex Order Average'])
        hexOrder = hexOrder.append(temp_df, ignore_index = True)

# Find the UHII of LA
openpath = "E:/Old Downloads Folder/Research Stuff/Climate Data/LA_UHII.csv"
with open(openpath) as f:
    UHII = pd.read_csv(f, sep = ',')

# Find the integral of the first peak from Ovito analyses
firstPeakIntegrals = pd.DataFrame()
for code in LA_codes:
    s1 = 'E:/Old Downloads Folder/Research Stuff/Jupyter Notebooks/Coordination Analysis Data/Los Angeles/'
    s2 = '_first_peak.txt'
    openpath = s1 + code + s2
    
    with open(openpath) as f:
        val = f.read()
        temp_df = pd.DataFrame([[code, float(val)]], columns = ['Station', 'First Peak Integral'])
        firstPeakIntegrals = firstPeakIntegrals.append(temp_df, ignore_index = True)

# This code finds the data found from QGIS and the average albedo of every area
albedos = pd.DataFrame()
for code in LA_codes:
    s1 = 'E:/Old Downloads Folder/Research Stuff/GIS Maps/Albedo Data/'
    s2 = '_weighted_albedo.txt'
    openpath = s1 + code + s2
    
    with open(openpath) as f:
        val = f.read()
        temp_df = pd.DataFrame([[code, float(val)]], columns = ['Station', 'Weighted Albedo'])
        albedos = albedos.append(temp_df, ignore_index = True)
        
# This code finds the clustering data from OVITO and finds the location of the first drop
drops = pd.DataFrame()
for code in LA_codes:
    s1 = 'E:/Old Downloads Folder/Research Stuff/Jupyter Notebooks/Coords Folder/Weather Station Coords/Los Angeles/First Drops/'
    s2 = '_first_drop.txt'
    openpath = s1 + code + s2
    
    with open(openpath) as f:
        val = f.read()
        temp_df = pd.DataFrame([[code, float(val)]], columns = ['Station', 'First Drop'])
        drops = drops.append(temp_df, ignore_index = True)

# This code finds the buildings per cluster at the location of the first drop
perCluster = pd.DataFrame()
for code in LA_codes:
    s1 = 'E:/Old Downloads Folder/Research Stuff/Jupyter Notebooks/Coords Folder/Weather Station Coords/First Drops/Buildings per Cluster/'
    s2 = '_buildings_per_cluster.txt'
    openpath = s1 + code + s2
    
    with open(openpath) as f:
        val = f.read()
        temp_df = pd.DataFrame([[code, float(val)]], columns = ['Station', 'Buildings Per Cluster'])
        perCluster = perCluster.append(temp_df, ignore_index = True)

# This code finds the data found from QGIS and the average albedo of every area
areaAlbedos = pd.DataFrame()
for code in LA_codes:
    s1 = 'E:/Old Downloads Folder/Research Stuff/GIS Maps/Albedo Data/'
    s2 = '_area_no_river_albedo.txt'
    openpath = s1 + code + s2
    
    with open(openpath) as f:
        val = f.read()
        temp_df = pd.DataFrame([[code, float(val)]], columns = ['Station', 'No River Albedo'])
        areaAlbedos = areaAlbedos.append(temp_df, ignore_index = True)

# This code finds the amount of buildings in the area around the weather station
totalBuildings = pd.DataFrame()
for code in LA_codes:
    s1 = 'E:/Old Downloads Folder/Research Stuff/Jupyter Notebooks/Coords Folder/Weather Station Coords/Los Angeles/'
    s2 = '_building_coords.txt'
    openpath = s1 + code + s2
    
    with open(openpath) as f:
        for i, l in enumerate(f):
            pass
    temp_df = pd.DataFrame([[code, int(i + 1)]], columns=['Station', 'Buildings In Area'])
    totalBuildings = totalBuildings.append(temp_df, ignore_index=True)

totalBuildings = totalBuildings.set_index(['Station'])
areaAlbedos = areaAlbedos.set_index(['Station'])
perCluster = perCluster.set_index(['Station'])
drops = drops.set_index(['Station'])
albedos = albedos.set_index(['Station'])
UHII = UHII.set_index(['STATION'])
firstPeakIntegrals = firstPeakIntegrals.set_index(['Station'])
hexOrder = hexOrder.set_index(['Station'])

In [157]:
# Pull all of our data into on frame
final_frame = pd.DataFrame()
final_frame = pd.concat([UHII, totalBuildings, areaAlbedos, perCluster, drops, albedos, firstPeakIntegrals, hexOrder], axis=1, sort=False)
final_frame

,UHI,Buildings In Area,No River Albedo,Buildings Per Cluster,First Drop,Weighted Albedo,First Peak Integral,Hex Order Average
USW00023130,2.369735,114805,0.44552,1822.301587,396.980,0.496099,1.841924,0.6575574
USW00093134,4.978082,173485,0.47755,2115.670732,348.867,0.522232,0.689432,0.657032


In [158]:
# Load up our predictors from Pittsburgh
path = "E:/Old Downloads Folder/Research Stuff/Jupyter Notebooks/Variable_Combos.csv"
with open(path) as f:
    predictors = pd.read_csv(f, sep=',')    
predictors = predictors.drop(['R2'], axis=1)

# Drop the rows with predictors we can't use. River things
bad_keys = []
for key, vals in predictors.iteritems():
    banned_vals = ['Area Albedo', 'Distance to River']
    index = 0
    for val in vals:
        val = str(val)
        val = val[:val.find(' -')]
        if val in banned_vals and index not in bad_keys:
            bad_keys.append(index)
        index += 1

# Reset the index
predictors = predictors.drop(bad_keys)
predictors = predictors.reset_index()
predictors = predictors.drop(['index'], axis=1)

# Run the predictions
LA_predictions = pd.DataFrame(columns=final_frame.index)
for code, data in final_frame.iterrows():
    all_predictions = []
    
    # Loops through the prediction frame row by row
    for index, row in predictors.iterrows():
        prediction = row['Intercept']
        for num in range(1, 4):
            col_name = 'Variable #' + str(num)
            coeff = row[col_name]
            if str(coeff) != 'nan':
                coeff_name = coeff[:coeff.find(' -')]
                coeff_val = float(coeff[coeff.find('- ') + 2:])
                
                LA_val = float(data[coeff_name])
                prediction += LA_val * coeff_val
        
        all_predictions.append(prediction)
    
    # Subtract the actual UHII from the predicted UHII
    LA_predictions[code] = [x - data['UHI'] for x in all_predictions]

LA_predictions['USW00023130'] = abs(LA_predictions['USW00023130']/final_frame['UHI'][0])
LA_predictions['USW00093134'] = abs(LA_predictions['USW00093134']/final_frame['UHI'][1])
LA_predictions = pd.concat([LA_predictions, predictors], axis=1, sort=False)

LA_predictions['Combined'] = (LA_predictions['USW00023130'] + LA_predictions['USW00093134'])/2
LA_predictions = LA_predictions.sort_values(by=['Combined'])
# LA_predictions = LA_predictions.reset_index
# LA_predictions = LA_predictions.drop(['index'])
LA_predictions

,USW00023130,USW00093134,Variable #1,Variable #2,Variable #3,Intercept,Combined
28,0.017253,0.018011,Weighted Albedo - 20.20835728690823,First Drop - 0.0021530825588092738,No River Albedo - 64.10558012681331,-37.029780,0.017632
44,0.043420,0.099169,Buildings In Area - 9.298497137373668e-06,No River Albedo - 46.03414816615081,Hex Order Average - 15.849490204512733,-29.525968,0.071294
6,0.161988,0.050854,Weighted Albedo - 3.0087517007158873,No River Albedo - 74.89879886753124,NaN,-32.107948,0.106421
25,0.097251,0.173751,First Peak Integral - -0.2627364065131408,No River Albedo - 52.37265096293579,Hex Order Average - 12.255147572572813,-28.768309,0.135501
0,0.221973,0.065452,No River Albedo - 75.18438725234495,NaN,NaN,-30.600396,0.143713
41,0.262009,0.038600,Buildings Per Cluster - -0.002046208658755087,Buildings In Area - 2.903415768451026e-05,No River Albedo - 60.37007965001221,-24.751692,0.150305
17,0.197655,0.106907,First Peak Integral - -0.39947312550935155,Weighted Albedo - 7.682289613613261,No River Albedo - 62.78506596688844,-28.209257,0.152281
21,0.224420,0.081864,First Peak Integral - -0.39332413485430834,Buildings Per Cluster - -0.00016342218822830052,No River Albedo - 64.89845565296446,-24.989732,0.153142
24,0.192567,0.158753,First Peak Integral - -0.020967779332784982,Buildings In Area - 2.3049340879129454e-05,Hex Order Average - 28.49961178159935,-18.521622,0.175660
13,0.196030,0.156314,Buildings In Area - 2.3530196376120226e-05,Hex Order Average - 28.71431917271189,NaN,-18.748422,0.176172


In [147]:
predictors

,Variable #1,Variable #2,Variable #3,Intercept
0,No River Albedo - 75.18438725234495,NaN,NaN,-30.600396
1,Hex Order Average - 55.0880980986435,NaN,NaN,-20.613677
2,First Peak Integral - -0.3899445914484401,No River Albedo - 63.792627556563495,NaN,-24.508852
3,First Peak Integral - 0.1522353770120916,Hex Order Average - 60.80192764755235,NaN,-23.474677
4,Weighted Albedo - 96.8164435644076,Buildings Per Cluster - 0.00796990071805326,NaN,-49.798417
5,Weighted Albedo - 72.0218528255336,Buildings In Area - 5.9562205905162044e-05,NaN,-37.800816
6,Weighted Albedo - 3.0087517007158873,No River Albedo - 74.89879886753124,NaN,-32.107948
7,Weighted Albedo - 18.50365197429314,Hex Order Average - 54.59904422462985,NaN,-30.467204
8,First Drop - 0.0010337593308137139,No River Albedo - 70.78608738173214,NaN,-28.825774
9,First Drop - 0.0017871877708474431,Hex Order Average - 49.59634764504227,NaN,-18.590526
